<a href="https://colab.research.google.com/github/maxMitsuya/webscraping/blob/main/Web_Scraping_de_Dados_da_Receita_Federal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Web Scraping de Dados da Receita Federal

Este projeto consiste em uma aplicação de web scraping desenvolvida utilizando a biblioteca Beautiful Soup 4. Foi criado como parte de um projeto prático realizado durante o curso na DNC. O código foi desenvolvido para ser executado diretamente no Google Colab, facilitando o acesso e a reprodução.

##Objetivo

O objetivo principal deste projeto é automatizar o processo de coleta de dados sobre empresas disponibilizados pela Receita Federal. A aplicação realiza o download dos dados diretamente do Banco de Dados da Receita Federal, organizados por data no formato "ano-mês" (onde o ano é representado por quatro dígitos e o mês por dois dígitos).

##Fonte dos dados

Os dados utilizados neste projeto são disponibilizados publicamente pela Receita Federal e podem ser acessados através do seguinte link:
[Banco de Dados da Receita Federal](https://arquivos.receitafederal.gov.br/cnpj
)

Os arquivos estão organizados por data, seguindo o padrão "ano-mês" (exemplo: 2023-10 para outubro de 2023).

##Funcionalidades

- Automatização do Download: O script realiza o
download automático dos arquivos de dados diretamente do site da Receita Federal.

- Organização por Data: Os dados são baixados e organizados de acordo com a data de publicação, facilitando a análise e o processamento posterior.

- Execução no Google Colab: O código foi desenvolvido para ser executado diretamente no Google Colab, permitindo que qualquer pessoa possa rodá-lo sem a necessidade de configurar um ambiente local.

##Carregando bibliotecas

In [ ]:
import os #Manipulação dos arquivos
import requests #Requisição de páginas web
from bs4 import BeautifulSoup #WebScraping
from urllib.parse import urljoin #Join de urls
from google.colab import drive #Montar o drive

##Montando o drive

In [ ]:
drive.mount('/content/drive')

In [ ]:
# Manter a conexão do google drive ativa
from google.colab import output
output.eval_js('google.colab.kernel.proxyPort(5000)')

Obs. Devido o volume de dados para realizar o download é interessante manter a conexão do drive para que não haja falhas.

##Função para baixar os dados

In [ ]:
def download_dados(url_base, data_coleta):

  # URL da página que contém os arquivos
  url = urljoin(url_base, data_coleta)

  # Diretório no Google Drive onde os arquivos serão salvos
  save_directory = "/content/drive/My Drive/" + data_coleta

  # Cria o diretório se ele não existir
  if not os.path.exists(save_directory):
      os.makedirs(save_directory)

  # Função para baixar um arquivo
  def download_file(file_url, file_name):
      print(f"Baixando {file_name}...")
      try:
          with requests.get(file_url, stream=True) as r:
              r.raise_for_status()
              with open(file_name, 'wb') as f:
                  for chunk in r.iter_content(chunk_size=8192):
                      f.write(chunk)
          print(f"{file_name} baixado com sucesso!")
      except Exception as e:
          print(f"Erro ao baixar {file_name}: {e}")

  # Faz a requisição para a página
  response = requests.get(url)
  response.raise_for_status()  # Verifica se a requisição foi bem-sucedida

  # Parseia o conteúdo HTML da página
  soup = BeautifulSoup(response.text, 'html.parser')

  # Lista para armazenar as URLs dos arquivos
  file_urls = []

  # Encontra todos os links na página
  for link in soup.find_all('a'):
      href = link.get('href')
      if href and href.endswith('.zip'):
          file_url = urljoin(url, href)
          file_name = os.path.join(save_directory, href.split('/')[-1])
          file_urls.append((file_url, file_name))

  # Baixa os arquivos em paralelo usando ThreadPoolExecutor
  with ThreadPoolExecutor(max_workers=5) as executor:  # Ajuste o número de workers conforme necessário
      executor.map(lambda args: download_file(*args), file_urls)

  print("Todos os arquivos foram baixados.")

##Função para definir a data de coleta

In [ ]:
from datetime import datetime, timedelta
def data(hoje):
  hoje = hoje
  mes_passado = hoje.replace(day=1) - timedelta(days=1)
  mes = mes_passado.month
  ano = mes_passado.year
  mes_str = '0' + str(mes) if mes < 10 else str(mes)
  data_coleta = f'{ano}' + '-' +mes_str + '/'
  return data_coleta

hoje = datetime.now()
data_coleta = data(hoje)

##Fazendo o download dos dados

In [ ]:
#Primeiro definir a url de coleta
url_base = "https://arquivos.receitafederal.gov.br/dados/cnpj/dados_abertos_cnpj/"

In [ ]:
#Baixa os dados usando a função download_dados()
download_dados(url_base, data_coleta)